In [ ]:
years = np.arange(2000,2020,1).tolist()

for year in years:
    web_name = 'https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html'
    website = request.urlopen(web_name).read()
    soup = BeautifulSoup(website,'lxml')

    table = soup.find('table', attrs={'class': 'sortable', 'id': 'per_game_stats'})
    table_headers = [header.text for header in table.find('thead').find_all('th')]
    table_rows = table.find_all('tr')

    final_data = []
    # For each table row,
    for tr in table_rows:
        # Make a list of the table data tags for this row
        td = tr.find_all('td')
        # Extract just the cell data and make a list
        row = [tr.text for tr in td]
        # Append the extracted data
        final_data.append(row)


    df = pd.DataFrame(final_data[1:], columns=table_headers[1:])
    df.to_csv('../data/nba_stats_'+str(year)+'.csv')
    
    time.sleep(1)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request
import time

In [2]:
url = request.urlopen('http://www.espn.com/nba/teams').read()
soup = BeautifulSoup(url,'lxml')

In [8]:
container = soup.find_all('div',{'class':'pl3'})

In [4]:
teamNames = [teamName.a.text for teamName in container]

In [5]:
teamUrls = [teamUrl.find_all('a',href=True)[1]['href'] for teamUrl in container]

In [6]:
teams = dict(zip(teamNames,teamUrls))

In [7]:
teams

{'Boston Celtics': '/nba/team/stats/_/name/bos/boston-celtics',
 'Brooklyn Nets': '/nba/team/stats/_/name/bkn/brooklyn-nets',
 'New York Knicks': '/nba/team/stats/_/name/ny/new-york-knicks',
 'Philadelphia 76ers': '/nba/team/stats/_/name/phi/philadelphia-76ers',
 'Toronto Raptors': '/nba/team/stats/_/name/tor/toronto-raptors',
 'Chicago Bulls': '/nba/team/stats/_/name/chi/chicago-bulls',
 'Cleveland Cavaliers': '/nba/team/stats/_/name/cle/cleveland-cavaliers',
 'Detroit Pistons': '/nba/team/stats/_/name/det/detroit-pistons',
 'Indiana Pacers': '/nba/team/stats/_/name/ind/indiana-pacers',
 'Milwaukee Bucks': '/nba/team/stats/_/name/mil/milwaukee-bucks',
 'Denver Nuggets': '/nba/team/stats/_/name/den/denver-nuggets',
 'Minnesota Timberwolves': '/nba/team/stats/_/name/min/minnesota-timberwolves',
 'Oklahoma City Thunder': '/nba/team/stats/_/name/okc/oklahoma-city-thunder',
 'Portland Trail Blazers': '/nba/team/stats/_/name/por/portland-trail-blazers',
 'Utah Jazz': '/nba/team/stats/_/name